# Задание на coursera #

## Линейная регрессия: прогноз оклада по описанию вакансии ##

Цель:
* использовать линейную регрессию
* применять линейную регрессию к текстовым данным

In [19]:
import numpy as np
import pandas as pd
from scipy.sparse import hstack
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

**Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах**

In [16]:
data = pd.read_csv('salary-train.csv')
data

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355
...,...,...,...,...
59995,"As a result of continued growth, First Class S...",Whitley Bay,contract,26400
59996,PHP / MVC Web Developer MacclesfieldCirca ***...,Macclesfield,permanent,26000
59997,"Staff Nurse, Nursing Home, Baldock White Recru...",Baldock,NaN,24500
59998,This is one of the best agency side opportunit...,The City,permanent,65000


**Проведите предобработку:**

In [17]:
def text_transform(text: pd.Series) -> pd.Series:
    return text.str.lower().replace("[^a-zA-Z0-9]", " ", regex=True)

**Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).**

In [20]:
vec = TfidfVectorizer(min_df=5)
X_train_text = vec.fit_transform(text_transform(data["FullDescription"]))

**Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'.**

In [21]:
data["LocationNormalized"].fillna("nan", inplace=True)
data["ContractTime"].fillna("nan", inplace=True)

**Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.**

In [22]:
enc = DictVectorizer()
X_train_cat = enc.fit_transform(data[["LocationNormalized", "ContractTime"]].to_dict("records"))

**Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.**

In [23]:
X_train = hstack([X_train_text, X_train_cat])

**Обучите гребневую регрессию с параметрами alpha=1 и random_state=241.**

In [24]:
y_train = data["SalaryNormalized"]
model = Ridge(alpha=1, random_state=241)
model.fit(X_train, y_train)

Ridge(alpha=1, random_state=241)

**Постройте прогнозы для двух примеров из файла salary-test-mini.csv.**

In [25]:
test = pd.read_csv("salary-test-mini.csv")
test

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [27]:
X_test_text = vec.transform(text_transform(test["FullDescription"]))
X_test_cat = enc.transform(test[["LocationNormalized", "ContractTime"]].to_dict("records"))
X_test = hstack([X_test_text, X_test_cat])

y_test = model.predict(X_test)
print(y_test)

[56581.31152328 37133.54332311]
